In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from googletrans import Translator

In [2]:
# generate a translated list of columns for every table in the data
data = Path('./vidas_data/')
tabs = pd.read_csv(data / 'table_info.csv')
tabs.head()

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,wHospital_FX_AI,wH_Spt,Sup_Complicanze_Postoperatorie,BASE TABLE
1,wHospital_FX_AI,wH_DirectTable,NutritionalRisk,BASE TABLE
2,wHospital_FX_AI,wH_Vitals,VitalDetection_Delete,BASE TABLE
3,wHospital_FX_AI,wH_DigitalSign,DigSigDoc_PubToRep,BASE TABLE
4,wHospital_FX_AI,wH_ADIManager,AdiConfiguration,BASE TABLE


In [3]:
tinfo = pd.read_csv(data/ 'row_and_column_count.csv')
tinfo = tinfo[tinfo.num_rows > 0].reset_index(drop=True)
tinfo['cols'] = ''
tinfo

,file_name,num_rows,num_columns,cols
0,wH_Conf_DBType,7,4,
1,wH_DirectTable_D_Scala_ESAS,3,5,
2,wH_Bck_IsWorking_Bck,1,16,
3,wH_Core_OperationalUnitAssignation,3284,12,
4,wH_Conf_OuHtmlHeader,21,18,
...,...,...,...,...
372,wH_MeshDiary_Configuration,2,12,
373,wH_Core_PHR_PlannedHospitalization,3233,34,
374,wH_Conf_CoreSec,8,7,
375,wH_DirectTable_C_Scala_ESAS,5,27,


In [4]:
df_cols = pd.DataFrame(data={'cols': [['A', 'B']]})
for idx, row in tinfo.iterrows():
    df_tmp = pd.read_csv(data /f'{row.file_name}.csv', low_memory=False)
    df_cols = df_cols.append({'cols': df_tmp.columns.tolist()}, ignore_index=True)
df_cols = df_cols.drop(df_cols.index[0]).reset_index(drop=True)
df_cols

,cols
0,"[IDDBType, TypeName, TypeDescriptionName, Order]"
1,"[IDScala_ESAS, Note_Deleted, Author_Insert, Ti..."
2,"[id, Internal_OUCode, Archive_Uri, Access_Cred..."
3,"[IDOperationalUnitAssignation, IDPatientHospit..."
4,"[IDOuHtmlHeader, IDOperationalUnit, IDHospital..."
...,...
372,"[IDConfiguration, GUIDOU, IDDictionary, Second..."
373,"[PhrId, PlannedEntryDate, PlannedPriority, Iss..."
374,"[IDCoreSec, ID_EHR_Definition, Name, Order, Lo..."
375,"[IDScala_ESAS, IDEHR, Author_Insert, TimeStamp..."


In [5]:
# save the tinfo dframe
tinfo['cols'] = df_cols
tinfo.to_csv('./vidas_data/table_cols_info.csv')
tinfo

,file_name,num_rows,num_columns,cols
0,wH_Conf_DBType,7,4,"[IDDBType, TypeName, TypeDescriptionName, Order]"
1,wH_DirectTable_D_Scala_ESAS,3,5,"[IDScala_ESAS, Note_Deleted, Author_Insert, Ti..."
2,wH_Bck_IsWorking_Bck,1,16,"[id, Internal_OUCode, Archive_Uri, Access_Cred..."
3,wH_Core_OperationalUnitAssignation,3284,12,"[IDOperationalUnitAssignation, IDPatientHospit..."
4,wH_Conf_OuHtmlHeader,21,18,"[IDOuHtmlHeader, IDOperationalUnit, IDHospital..."
...,...,...,...,...
372,wH_MeshDiary_Configuration,2,12,"[IDConfiguration, GUIDOU, IDDictionary, Second..."
373,wH_Core_PHR_PlannedHospitalization,3233,34,"[PhrId, PlannedEntryDate, PlannedPriority, Iss..."
374,wH_Conf_CoreSec,8,7,"[IDCoreSec, ID_EHR_Definition, Name, Order, Lo..."
375,wH_DirectTable_C_Scala_ESAS,5,27,"[IDScala_ESAS, IDEHR, Author_Insert, TimeStamp..."


In [6]:
# get a one to one translation of every unique column in the data
f_to_col = {}
for idx, e in tinfo.iterrows():
    f_to_col[e.file_name] = (~pd.read_csv(data/f'{e.file_name}.csv', low_memory=False).isna()).sum() if e.num_rows > 0 else None
df = pd.DataFrame(f_to_col).transpose().reset_index()
df.columns = ['file_name'] + list(df.columns[1:])
df

,file_name,AMT,API_Version,AREADIAGNOSTICA,ASL,ATCCode,ATCDescription,Abbigliamento,Abitazione,AccessId,...,visioni_morte,visioni_vita,volonta,vomito_occasionale,vomito_persistente,vulva,wHCode,wHKey,wHRole,wHValue
0,wH_Conf_DBType,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,wH_DirectTable_D_Scala_ESAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wH_Bck_IsWorking_Bck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,wH_Core_OperationalUnitAssignation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,wH_Conf_OuHtmlHeader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,wH_MeshDiary_Configuration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,wH_Core_PHR_PlannedHospitalization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,wH_Conf_CoreSec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,wH_DirectTable_C_Scala_ESAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# save the dframe giving a cross-tabulation between table name and unique columns
df.to_csv('./vidas_data/cross_ref_cols_tabs.csv', index=False)

In [8]:
translator = Translator()
df_col_trans = pd.DataFrame({'IT': df.columns.tolist(), 'inter': '', 'EN': ''})
df_col_trans['inter'] = df_col_trans['IT']
df_col_trans['EN'] = df_col_trans['inter'].apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))
df_col_trans

,IT,inter,EN
0,file_name,file_name,file_name
1,AMT,AMT,AMT
2,API_Version,API_Version,API_Version
3,AREADIAGNOSTICA,AREADIAGNOSTICA,AREADIAGNOSTICS
4,ASL,ASL,ASL
...,...,...,...
2479,vulva,vulva,vulva
2480,wHCode,wHCode,wHCode
2481,wHKey,wHKey,wHKey
2482,wHRole,wHRole,wHRole


In [12]:
# save the dframe
df_col_trans = df_col_trans.drop(columns='inter')
df_col_trans.to_csv('./vidas_data/col_translated_ref.csv', index=False)

In [2]:
# translations experiment on drug names
df_main = pd.read_csv('./vidas_data/wH_Therapy_JCM_Therapy.csv')

In [3]:
df_main.columns

Index(['IDTherapy', 'TherapyType', 'IDCorrelatedTherapy', 'IDEHR',
       'DateTherapy', 'IDDrug', 'DrugCode', 'DrugXMLIdentifier',
       'DrugIsAutoVirtual', 'DrugEANCode', 'DrugName', 'ATCCode',
       'ATCDescription', 'CCTypeOfUsageCode', 'ComplementaryCodeTypeOfUsage',
       'ComplementaryCodeDoseCode', 'ComplementaryCodeDose',
       'ActiveIngredientCode', 'ActiveIngredientDescription', 'DrugGrammatura',
       'DrugShapeDescription', 'DrugPackagingShapeDescription', 'Dose',
       'DayFrequency', 'HoursString', 'TypeOfUsage', 'DOM', 'ToBeReplanned',
       'StartTomorrow', 'SuspensionDay', 'MultiDay', 'Immediate',
       'HasDoseSchema', 'DoseSchemaDescr', 'DoseSchemaXML', 'Note',
       'Author_Insert', 'TimeStamp_Insert', 'IDDigitalSignDocument'],
      dtype='object')

In [4]:
df_main.head()

,IDTherapy,TherapyType,IDCorrelatedTherapy,IDEHR,DateTherapy,IDDrug,DrugCode,DrugXMLIdentifier,DrugIsAutoVirtual,DrugEANCode,...,SuspensionDay,MultiDay,Immediate,HasDoseSchema,DoseSchemaDescr,DoseSchemaXML,Note,Author_Insert,TimeStamp_Insert,IDDigitalSignDocument
0,1,Generica,NaN,2,2015-01-09 00:00:00.000,NaN,024402063,"<DrugSearchItem Type=""RealDrug"" Identifier=""10...",False,NaN,...,False,False,False,False,125 mcg,NaN,dal 12 gennaio 2015,Lonati Giada Carla - LNTGCR68S56F205H,2015-01-09 16:06:16.677,1185
1,2,Generica,NaN,2,2015-01-09 00:00:00.000,NaN,010089035,"<DrugSearchItem Type=""RealDrug"" Identifier=""46...",False,NaN,...,False,False,False,False,25 mg,NaN,dal 12 gennaio 2015,Lonati Giada Carla - LNTGCR68S56F205H,2015-01-09 16:06:16.677,1186
2,3,Generica,NaN,2,2015-01-09 00:00:00.000,NaN,034216186,"<DrugSearchItem Type=""RealDrug"" Identifier=""47...",False,NaN,...,False,False,False,False,20 mg,NaN,dal 12 gennaio 2015,Lonati Giada Carla - LNTGCR68S56F205H,2015-01-09 16:06:16.677,1187
3,4,Generica,NaN,2,2015-01-09 00:00:00.000,NaN,020766046,"<DrugSearchItem Type=""RealDrug"" Identifier=""72...",False,NaN,...,False,False,False,False,10 mg,NaN,dal 12 gennaio 2015,Lonati Giada Carla - LNTGCR68S56F205H,2015-01-09 16:06:16.677,1188
4,5,Generica,NaN,2,2015-01-09 00:00:00.000,NaN,012745182,"<DrugSearchItem Type=""RealDrug"" Identifier=""53...",False,NaN,...,False,False,False,False,1 g,NaN,dal 12 gennaio 2015,Lonati Giada Carla - LNTGCR68S56F205H,2015-01-09 16:06:16.677,1189


In [5]:
df_main.DrugName.value_counts()

SOLDESAM              18257
MORFINA CL MOLT.      13413
SERENASE              11092
LASIX                 10021
MOVICOL                9106
                      ...  
FLIXOTIDE DISK 500        1
CEFTRIAXONE KABI          1
MORFINA CL MONICO         1
ONDANSETRONE MGI          1
EXEMESTANE TEVA           1
Name: DrugName, Length: 1214, dtype: int64

In [6]:
df_drug = df_main[['IDEHR', 'DateTherapy', 'DrugName', 'ActiveIngredientDescription', 'Dose']]
df_drug.head(20)

,IDEHR,DateTherapy,DrugName,ActiveIngredientDescription,Dose
0,2,2015-01-09 00:00:00.000,EUTIROX,LEVOTIROXINA SODICA,125 mcg
1,2,2015-01-09 00:00:00.000,DELTACORTENE,PREDNISONE,25 mg
2,2,2015-01-09 00:00:00.000,PARIET,RABEPRAZOLO SODICO,20 mg
3,2,2015-01-09 00:00:00.000,PLASIL,METOCLOPRAMIDE CLORIDRATO,10 mg
4,2,2015-01-09 00:00:00.000,TACHIPIRINA,PARACETAMOLO,1 g
5,2,2015-01-09 00:00:00.000,TRIATEC,RAMIPRIL,2.5 mg
6,2,2015-01-09 00:00:00.000,MIRTAZAPINA EG,MIRTAZAPINA,15 mg
7,2,2015-01-09 00:00:00.000,MOVICOL,POTASSIO CLORURO\SODIO CLORURO\SODIO BICARBONA...,1 b
8,2,2015-01-09 00:00:00.000,PURSENNID,SENNA FOGLIA (GLUCOSIDI PURI),1 cpr
9,1020,2015-01-10 00:00:00.000,OMEPRAZOLO TEVA,OMEPRAZOLO,20 mg


In [7]:
df_tmp = df_drug[0:2000]
df_tmp

,IDEHR,DateTherapy,DrugName,ActiveIngredientDescription,Dose
0,2,2015-01-09 00:00:00.000,EUTIROX,LEVOTIROXINA SODICA,125 mcg
1,2,2015-01-09 00:00:00.000,DELTACORTENE,PREDNISONE,25 mg
2,2,2015-01-09 00:00:00.000,PARIET,RABEPRAZOLO SODICO,20 mg
3,2,2015-01-09 00:00:00.000,PLASIL,METOCLOPRAMIDE CLORIDRATO,10 mg
4,2,2015-01-09 00:00:00.000,TACHIPIRINA,PARACETAMOLO,1 g
...,...,...,...,...,...
1995,1019,2015-02-02 00:00:00.000,DIFLUCAN,FLUCONAZOLO,200 mg
1996,1043,2015-01-24 00:00:00.000,BUSCOPAN,SCOPOLAMINA BUTILBROMURO,20 mg
1997,1043,2015-01-24 00:00:00.000,SERENASE,ALOPERIDOLO,2 mg
1998,1043,2015-01-24 00:00:00.000,UGUROL,ACIDO TRANEXAMICO,500 mg


In [11]:
# checking for NaNs
print(sum(df_tmp.DrugName.isna()))
print(sum(df_tmp.ActiveIngredientDescription.isna()))

0
33


In [12]:
df_tmp['DrugName_Translated'] = df_tmp['DrugName'].apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))
df_tmp

<ipython-input-12-f8760f8fbb42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['DrugName_Translated'] = df_tmp['DrugName'].apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))


,IDEHR,DateTherapy,DrugName,ActiveIngredientDescription,Dose,DrugName_Translated
0,2,2015-01-09 00:00:00.000,EUTIROX,LEVOTIROXINA SODICA,125 mcg,EUTIROX
1,2,2015-01-09 00:00:00.000,DELTACORTENE,PREDNISONE,25 mg,DELTACORTENE
2,2,2015-01-09 00:00:00.000,PARIET,RABEPRAZOLO SODICO,20 mg,PARIET
3,2,2015-01-09 00:00:00.000,PLASIL,METOCLOPRAMIDE CLORIDRATO,10 mg,PLASIL
4,2,2015-01-09 00:00:00.000,TACHIPIRINA,PARACETAMOLO,1 g,TACHIPIRINA
...,...,...,...,...,...,...
1995,1019,2015-02-02 00:00:00.000,DIFLUCAN,FLUCONAZOLO,200 mg,DIFLUCAN
1996,1043,2015-01-24 00:00:00.000,BUSCOPAN,SCOPOLAMINA BUTILBROMURO,20 mg,BUSCOPAN
1997,1043,2015-01-24 00:00:00.000,SERENASE,ALOPERIDOLO,2 mg,SERENASE
1998,1043,2015-01-24 00:00:00.000,UGUROL,ACIDO TRANEXAMICO,500 mg,UGUROL


In [13]:
df_tmp['ActiveIng_Translated'] = df_tmp['ActiveIngredientDescription'].dropna().apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))
df_tmp

<ipython-input-13-911514a234ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['ActiveIng_Translated'] = df_tmp['ActiveIngredientDescription'].dropna().apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))


,IDEHR,DateTherapy,DrugName,ActiveIngredientDescription,Dose,DrugName_Translated,ActiveIng_Translated
0,2,2015-01-09 00:00:00.000,EUTIROX,LEVOTIROXINA SODICA,125 mcg,EUTIROX,LEVOTHYROXIN SODIUM
1,2,2015-01-09 00:00:00.000,DELTACORTENE,PREDNISONE,25 mg,DELTACORTENE,PREDNISONE
2,2,2015-01-09 00:00:00.000,PARIET,RABEPRAZOLO SODICO,20 mg,PARIET,RABEPRAZOLE SODIUM
3,2,2015-01-09 00:00:00.000,PLASIL,METOCLOPRAMIDE CLORIDRATO,10 mg,PLASIL,METOCLOPRAMIDE HYDROCHLORIDE
4,2,2015-01-09 00:00:00.000,TACHIPIRINA,PARACETAMOLO,1 g,TACHIPIRINA,PARACETAMOL
...,...,...,...,...,...,...,...
1995,1019,2015-02-02 00:00:00.000,DIFLUCAN,FLUCONAZOLO,200 mg,DIFLUCAN,FLUCONAZOLE
1996,1043,2015-01-24 00:00:00.000,BUSCOPAN,SCOPOLAMINA BUTILBROMURO,20 mg,BUSCOPAN,SCOPOLAMINA BUTYLBROMIDE
1997,1043,2015-01-24 00:00:00.000,SERENASE,ALOPERIDOLO,2 mg,SERENASE,ALOPERIDOLO
1998,1043,2015-01-24 00:00:00.000,UGUROL,ACIDO TRANEXAMICO,500 mg,UGUROL,TRANEXAMIC ACID


In [14]:
df_drug.shape

(292176, 5)

In [62]:
# df_drug['DrugName_Translated'] = df_drug['DrugName'].apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))
df_drug.head(10)

NameError: name 'df_drug' is not defined

In [ ]:
df_drug['ActiveIng_Translated'] = df_drug['ActiveIngredientDescription'].dropna().apply(translator.translate, src='it', dest='en').apply(getattr, args=('text',))
df_drug.head(10)

In [ ]:
df_drug.to_pickle('./hospiedata/drug_trans.csv')